In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Your API Key'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('companies.csv')

In [4]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN


In [5]:
from vertexai.language_models import TextGenerationModel

In [6]:
llm = TextGenerationModel.from_pretrained('text-bison')

In [10]:
llm.predict(prompt="tell me about the company Minio", max_output_tokens=2048)

 Minio is a high-performance, distributed object storage platform built on an open-source foundation. It is designed to be scalable, reliable, and secure, making it suitable for a wide range of use cases, including cloud storage, data backup, and disaster recovery.

Minio is written in Go and is based on the Amazon S3 API, making it compatible with a wide range of existing tools and applications. It offers a variety of features, including:

* **Scalability:** Minio can be scaled to support petabytes of data and thousands of concurrent users.
* **Reliability:** Minio uses erasure coding to protect data from hardware failures, and it can be configured to replicate data across multiple servers for added redundancy.
* **Security:** Minio supports a variety of security features, including encryption at rest, access control lists, and multi-factor authentication.
* **Cost-effectiveness:** Minio is open-source and free to use, making it a cost-effective alternative to commercial object storag

In [24]:
import ast
#this is to create a list
ast.literal_eval(df.iloc[2]['Investors'])


#df[df['Company'] == 'Minio']['Investors']



'Dell Technologies Capital'

## hallucinations mixed with true information, give model some context with private data

In [28]:
def summary(company, crunchbase_url, city, country, industry, investor_list):
    investors = "The investors in the company are"
    for investor in ast.literal_eval(investor_list):
        investors += f" {investor}, "
        
    text = f"{company} has headquarters in {city} in {country} and is in the field of {industry}. {investors}. You can find more information here: {crunchbase_url}."
    return text

In [29]:
df['summary'] = df.apply(lambda df: summary(df['Company'], df['Crunchbase Url'], df['City'], df['Country'], df['Industry'], df['Investors']), axis=1)

In [30]:
df.head()

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.0,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.0,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.0,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.0,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.0,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...


In [33]:
context = df['summary'].iloc[2]

In [34]:
prompt = f'tell me about the company Minio, here is some context:\n{context}'

In [35]:
llm.predict(prompt, max_output_tokens=2048, temperature = 0)

 **Minio** is a leading provider of open-source object storage software. Founded in 2014, the company is headquartered in Palo Alto, California, with offices in Bangalore, India, and London, UK. Minio's mission is to make object storage simple, scalable, and cost-effective for businesses of all sizes.

Minio's flagship product, MinIO, is a high-performance, distributed object storage system that is designed to be easy to use and manage. MinIO is built on a microservices architecture, which makes it highly scalable and resilient. It also supports a wide range of features, including data replication, encryption, and access control.

Minio has a strong customer base that includes some of the world's largest companies, such as Google, Uber, and Airbnb. The company has also been recognized by industry analysts, such as Gartner and Forrester, for its innovative technology and strong market presence.

In 2021, Minio raised $40 million in a Series B funding round led by General Catalyst. The c

## automate this process by cosin similarities on summary and create text embeddings inject it as context for LLM

In [36]:
from vertexai.language_models import TextEmbeddingModel

In [37]:
embedder = TextEmbeddingModel.from_pretrained('textembedding-gecko@001')

In [38]:
embeddings = embedder.get_embeddings(['Hello my name is Ran'])

In [41]:
len(embeddings[0].values)
#768 dimensions

768

In [42]:
df['summary']

0       Esusu has headquarters in New York in United S...
1       Fever Labs has headquarters in New York in Uni...
2       Minio has headquarters in Palo Alto in United ...
3       Darwinbox has headquarters in Hyderabad in Ind...
4       Pentera has headquarters in Petah Tikva in Isr...
                              ...                        
1194    Fanatics has headquarters in Jacksonville in U...
1195    SpaceX has headquarters in Hawthorne in United...
1196    Vice Media has headquarters in Brooklyn in Uni...
1197    Klarna has headquarters in Stockholm in Sweden...
1198    Veepee has headquarters in La Plaine Saint-Den...
Name: summary, Length: 1199, dtype: object

In [43]:
new_df = df.head(10)

In [44]:
def get_summary_embeddings(summary):
    return embedder.get_embeddings([summary])[0].values

In [45]:
get_summary_embeddings(df['summary'][0])

[-0.030026588588953018,
 0.005382542498409748,
 -0.001747897476889193,
 0.0028557111509144306,
 0.05501573532819748,
 -0.07258821278810501,
 0.010174637660384178,
 -0.023739131167531013,
 -0.027584783732891083,
 0.03434348851442337,
 0.05031266435980797,
 0.06254731863737106,
 0.05042867735028267,
 0.01616852357983589,
 0.004203496500849724,
 -0.016123145818710327,
 -0.032454293221235275,
 -0.05189991742372513,
 0.040731366723775864,
 0.015099425800144672,
 -0.04136137291789055,
 -0.024378051981329918,
 -0.022744810208678246,
 0.016293339431285858,
 -0.006314689759165049,
 -0.05059725418686867,
 0.006125611253082752,
 0.008889065124094486,
 -0.011463264003396034,
 0.03484640270471573,
 0.012790960259735584,
 0.02009129524230957,
 -0.06840696185827255,
 -0.0027328594587743282,
 0.007849759422242641,
 0.024744516238570213,
 -0.036392293870449066,
 0.02645087242126465,
 -0.028730006888508797,
 0.01257738471031189,
 0.0002939972036983818,
 -0.028062377125024796,
 0.05056503042578697,
 -0.0

In [46]:
#new_df['embeddings'] = new_df.apply(lambda new_df: get_summary_embeddings(new_df['summary']), axis=1)

/var/folders/v7/4l89c8s95blcs1_nz9glhysdcwkd15/T/ipykernel_76198/1286484162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['embeddings'] = new_df.apply(lambda new_df: get_summary_embeddings(new_df['summary']), axis=1)


In [47]:
new_df['embeddings'] = new_df['summary'].apply(get_summary_embeddings)

/var/folders/v7/4l89c8s95blcs1_nz9glhysdcwkd15/T/ipykernel_76198/3208643835.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['embeddings'] = new_df['summary'].apply(get_summary_embeddings)


In [48]:
new_df

,Updated at,Company,Crunchbase Url,Last Valuation (Billion $),Date Joined,Year Joined,City,Country,Industry,Investors,Company Website,summary,embeddings
0,"10/31/2022, 2:37:05 AM",Esusu,https://www.cbinsights.com/company/esusu,1.00,1/27/2022,2022,New York,United States,Fintech,"[""Next Play Ventures"",""Zeal Capital Partners"",...",NaN,Esusu has headquarters in New York in United S...,"[-0.030026588588953018, 0.005382542498409748, ..."
1,"10/31/2022, 2:37:05 AM",Fever Labs,https://www.cbinsights.com/company/fever-labs,1.00,1/26/2022,2022,New York,United States,Internet software & services,"[""Accel"",""14W"",""GS Growth""]",NaN,Fever Labs has headquarters in New York in Uni...,"[0.0326564721763134, -0.015538864769041538, -0..."
2,"10/31/2022, 2:37:04 AM",Minio,https://www.cbinsights.com/company/minio,1.00,1/26/2022,2022,Palo Alto,United States,Data management & analytics,"[""General Catalyst"",""Nexus Venture Partners"",""...",NaN,Minio has headquarters in Palo Alto in United ...,"[0.014229566790163517, -0.03516090288758278, 0..."
3,"10/31/2022, 2:37:04 AM",Darwinbox,https://www.cbinsights.com/company/darwinbox,1.00,1/25/2022,2022,Hyderabad,India,Internet software & services,"[""Lightspeed India Partners"",""Sequoia Capital ...",NaN,Darwinbox has headquarters in Hyderabad in Ind...,"[0.017399685457348824, -0.02392030879855156, -..."
4,"10/31/2022, 2:37:04 AM",Pentera,https://www.cbinsights.com/company/pcysys,1.00,1/11/2022,2022,Petah Tikva,Israel,Cybersecurity,"[""AWZ Ventures"",""Blackstone"",""Insight Partners""]",NaN,Pentera has headquarters in Petah Tikva in Isr...,"[-0.0210246704518795, -0.01949070207774639, 0...."
5,"10/31/2022, 2:37:04 AM",Placer.ai,https://www.cbinsights.com/company/placerai,1.00,1/12/2022,2022,Los Altos,United States,Artificial intelligence,"[""Fifth Wall Ventures"",""JBV Capital"",""Array Ve...",NaN,Placer.ai has headquarters in Los Altos in Uni...,"[-0.006333084776997566, -0.011408934369683266,..."
6,"10/31/2022, 2:36:06 AM",CAIS,https://www.cbinsights.com/company/cais,1.10,1/11/2022,2022,New York,United States,Fintech,"[""Franklin Templeton"",""Motive Partners. Apollo...",NaN,CAIS has headquarters in New York in United St...,"[-0.0038531150203198195, -0.016784440726041794..."
7,"10/31/2022, 2:36:04 AM",Chief,https://www.cbinsights.com/company/chief-1,1.10,3/31/2022,2022,New York,United States,Other,"[""General Catalyst"",""Inspired Capital"",""Flybri...",NaN,Chief has headquarters in New York in United S...,"[-0.007303301244974136, -0.0232907272875309, 0..."
8,"10/31/2022, 2:34:41 AM",Ankorstore,https://www.cbinsights.com/company/ankorstore,1.98,1/10/2022,2022,Paris,France,E-commerce & direct-to-consumer,"[""Global Founders Capital"",""Aglae Ventures"",""A...",NaN,Ankorstore has headquarters in Paris in France...,"[-0.024535024538636208, -0.0074149612337350845..."
9,"10/31/2022, 2:37:16 AM",Unstoppable Domains,https://www.cbinsights.com/company/unstoppable...,1.00,7/27/2022,2022,Las Vegas,United States,Internet software & services,"[""Boost VC"",""Draper Associates"",""Gaingels""]",NaN,Unstoppable Domains has headquarters in Las Ve...,"[-0.017770864069461823, -0.041798632591962814,..."


In [49]:
import numpy as np

In [51]:
def vector_similarity(A,B):
    vec1 = np.array(A)
    vec2 = np.array(B)
    
    return np.dot(np.array(vec1), np.array(vec2))

In [52]:
vector_similarity(new_df.iloc[0]['embeddings'], new_df.iloc[1]['embeddings'])

0.8129039746428122

In [58]:
def embed_prompt_lookup():
    question = input('What question do you have about a startup?\n')
    
    #API call --> embedding the input
    prompt_embedding = get_summary_embeddings(question)
    
    #SIMILARITY SEARCH
    new_df['prompt_similarity'] = new_df['embeddings'].apply(lambda vector: vector_similarity(vector,prompt_embedding))
    
    #grab the most similar array
    #new_df[new_df['prompt_similarity'] == new_df['prompt_similarity'].max()]['summary']
    context = new_df.nlargest(1, 'prompt_similarity').iloc[0]['summary']
    
    #prompt + context injection
    prompt=f"""Only answer the question below if you have 100% certainty of the facts. Use the context below to answer.
    Here are some context:
    {context}
    Question:
    {question}
    """
    
    result = llm.predict(prompt=prompt, max_output_tokens=2048, temperature=0)
    
    print(result.text)

In [60]:
embed_prompt_lookup()

What question do you have about a startup?
Tell me about Esusu startup


/var/folders/v7/4l89c8s95blcs1_nz9glhysdcwkd15/T/ipykernel_76198/2769443427.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['prompt_similarity'] = new_df['embeddings'].apply(lambda vector: vector_similarity(vector,prompt_embedding))


 Esusu is a fintech company headquartered in New York, United States. It provides a platform that enables individuals to build credit by reporting their rent payments to credit bureaus. The company was founded in 2018 and has raised over $13 million in funding from investors including Next Play Ventures, Zeal Capital Partners, and SoftBank Group.
